# Chapter 14

Word Representations/word embeddings

word cooccurrence matrix/
term-term Matrix/
Term-Context matrix


Window-size/context-size

example window-size 2
check words 2 left and 2 right
XXX XXX Cherry XXX XXX





In [1]:
import numpy as np
import re

doc = open('../data/azureDevOps.txt',encoding="utf8").read().lower()

doclen = len(doc)

corpus_list = re.split('\W+',doc)

corpuslen = len(corpus_list)

print(doclen)
print(corpuslen)

358064
59077


In [2]:
cutOffValue = 100
from collections import defaultdict
frequency = defaultdict(int)
for token in corpus_list:
    frequency[token] += 1
processedCorpus_list = [token for token in corpus_list 
                        if frequency[token] >= cutOffValue]

len(processedCorpus_list)

34004

In [3]:
allWords = np.array(list(frequency.keys()))
allCounts = np.array(list(frequency.values()))

vocab = allWords[allCounts >= cutOffValue]
wordCounts = allCounts[allCounts >= cutOffValue]

len(vocab)

108

In [4]:
from scipy.sparse import lil_matrix

def computeWordContextMatrix(corpus_list,vocab=None,windowSize=2):
    if vocab is None:
        vocab = sorted(list(set(cospus_list)))
    numWords = len(vocab)
    M = np.zeros((numWords,numWords))
    #M = lil_matrix((numWords,numWords))#use for large matrix put takes longer
    W2I = dict(zip(vocab,np.arange(numWords)))
    I2W = dict(zip(np.arange(numWords),vocab))
    doc = corpus_list
    curIdx = 0
    docLen = len(doc)
    while curIdx < docLen:
        left = max(curIdx-windowSize,0)
        right = min(curIdx+windowSize+1,docLen)
        wordsInContext = doc[left:curIdx] + doc[curIdx+1:right]
        currentWord = doc[curIdx]
        currentWordIdx = W2I[currentWord]
        for word in wordsInContext:
            contextWordIdx = W2I[word]
            M[currentWordIdx,contextWordIdx] += 1
        curIdx += 1
    return M,W2I,I2W

M,W2I,I2W = computeWordContextMatrix(processedCorpus_list,vocab)
M.shape

(108, 108)

In [5]:
len(vocab)

print(vocab)

['azure' 'devops' 'with' 'and' 'your' 'all' 'of' 'this' 'be' 'in' 'a' 'or'
 'by' 'the' 'to' 'is' 's' 'will' 'for' 'have' 'use' 'development'
 'project' '1' '3' 'com' 'our' 'as' 'you' 'from' '4' 'that' 'can' 'at'
 'are' 'on' 'also' 'up' 'microsoft' 'an' 'it' 'now' 'work' 'using'
 'application' '6' 'new' 'if' 'we' 'section' '2' 'create' '5' 'ci' 'cd'
 'control' 'code' 'boards' 'pipelines' 'test' 'artifacts' 'creating'
 'organization' 'items' 'source' 'github' 'repository' 'pull' 'request'
 'branch' 'visual' 'studio' 'release' 'build' 'agents' 'hosted' 'when'
 'agent' 'pipeline' 'stage' 'container' 'testing' 'setting' 'set'
 'package' 'deployment' 'based' 'service' 'chapter' 'how' 'add' 'need'
 'following' 'https' 'next' 'used' 'here' 'run' 'name' 'see' 'select'
 'then' 'different' 'figure' 'll' 'created' 'task' 'click']


In [6]:
word = 'azure'
print(W2I[word],I2W[0])

0 azure


In [7]:
v = M[W2I['azure'],:]
print(v)

[164. 772. 342. 202. 128.   7.  56.  26.  16. 257.  82.  30.  30. 250.
 208.  52.  20.  17. 106.  15.  31.  11.  46.  44.  18. 102.  10.  73.
  89.  45.  38.  26.  49.  14.  22.  52.   4.  42.  88.  88.  13.  12.
  21.  64.  17.  27.   5.   6.  33.   9.  32.  23.  20.  19.  50.  56.
  19. 124. 251.  76.  49.  51.  42.  11.   5.  69.  19.  10.   5.   7.
   2.   9.  10.  14.  39.  22.  10. 121. 167.   6.  39.  10.  33.  15.
   2.  20.   8.  54.  36.   8.   3.   6.   8.  44.   5.  13.   5.  10.
   5.   5.  29.   4.  12.  61.   5.   6.  15.  19.]


In [8]:
v.shape

(108,)

In [9]:
def pmi(M, positive=True):
    col_totals = np.sum(M,axis=0)
    total = col_totals.sum()
    row_totals = np.sum(M,axis=1)
    expected = np.outer(row_totals, col_totals) / total
    M = M / expected    
    with np.errstate(divide='ignore'):
        M = np.log(M)
    M[np.isinf(M)] = 0.0  
    if positive:
        M[M < 0] = 0.0
    return M

M = pmi(M)
M[W2I['azure'],:]

array([0.        , 1.77202695, 1.24628025, 0.        , 0.33224931,
       0.        , 0.        , 0.        , 0.        , 0.30914361,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.50177976, 0.        ,
       1.52111802, 0.        , 0.15002775, 0.        , 0.        ,
       0.32421406, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.36510865, 1.07644534, 0.63981364,
       0.        , 0.        , 0.        , 0.24493384, 0.        ,
       0.372801  , 0.        , 0.        , 0.        , 0.        ,
       0.10473576, 0.        , 0.09087872, 0.        , 0.7640434 ,
       1.1967255 , 0.        , 1.73924145, 1.70982845, 0.32421406,
       0.96878443, 0.3338249 , 0.90904343, 0.        , 0.        ,
       0.39658333, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.52280

In [10]:
from sklearn.decomposition import TruncatedSVD,PCA,IncrementalPCA
transformer = TruncatedSVD(n_components=100)
M_reduced = transformer.fit_transform(M)
M_reduced.shape
M.shape
M_reduced[W2I['azure'],:]

array([ 1.94901798, -1.59300668,  0.71913953,  1.12980366,  1.59773673,
        0.38685175,  1.28525832,  0.62673846,  0.18142335,  0.30662337,
       -0.1308667 , -0.36504702,  0.4821801 , -0.09906275,  0.13812073,
       -0.10821644,  0.28443539, -0.43743388,  0.30639659, -0.60988739,
       -0.41023503,  0.19680526, -0.14679521,  0.81943244, -1.11180521,
       -0.3513152 , -1.12093386,  0.10924543, -0.14510302,  0.88124274,
        0.1707201 ,  0.41344565,  0.39674479, -0.09485438,  0.30663128,
       -0.87749262, -0.16830552, -0.38545493, -0.23450951,  0.8415997 ,
        0.32758856,  0.48751807, -0.49156696,  0.11984287, -0.30241775,
        0.13327269, -0.54191605, -0.38719888, -0.31506158, -0.24696457,
        0.19810125,  0.17332525,  0.30934387, -0.0890748 ,  0.14468264,
        0.07520589, -0.32599125,  0.56718235,  0.22568295,  0.18819536,
        0.04231654, -0.47154537, -0.0146015 , -0.3009912 ,  0.02071921,
        0.01962504,  0.1051733 , -0.2679721 ,  0.17405856, -0.00

In [11]:
def getNorms(E):
    if E.ndim == 1:
        E = E[np.newaxis,:]
    nrms = np.sum(E**2,axis=1)**0.5
    return nrms

def normalize(E):
    if E.ndim == 1:
        E = E[np.newaxis,:]
    nrms = getNorms(E)
    return E/nrms[:,np.newaxis]

def cosineSimilarity(E,v):
    E = normalize(E)
    v = normalize(v)
    scores = E.dot(v.T)
    return scores

def getMostSimilarWords(E,word,W2I,topn=10):
    if type(word) is str:
        v = E[W2I[word],:]
    else:
        v = word
    scores = cosineSimilarity(E,v)
    scores = scores.squeeze()
    sortedScores = np.sort(scores)[::-1]
    idx = np.argsort(scores)[::-1]
    topNScores = sortedScores[:topn]
    topNWordsIdx = idx[:topn]
    return topNScores,topNWordsIdx

In [15]:
scores,idx = getMostSimilarWords(M_reduced,'machine',W2I)
for i in range(len(idx)):
    print(scores[i],I2W[idx[i]])

KeyError: 'machine'

In [13]:
print(getNorms(M_reduced[[1,4,5],:]))
v.shape

import dill
def saveModel(model,fileNameDotpkl):
    with open(fileNameDotpkl,'wb') as f:
        dill.dump(model,f)

model = M_reduced,W2I,I2W,vocab;
saveModel(model,'../data/w2v_100d.pkl')

[3.72737092 3.54336143 3.19724049]
